In [27]:
import pathlib
from bs4 import BeautifulSoup
from IPython.display import clear_output
import pandas as pd
import datetime
import pickle
import requests
import time

In [2]:
table = pd.read_csv(r'D:\股票網頁\股票\價量\上櫃_csv\2007-01-02.csv')

root = pathlib.Path('D:\股票網頁\股票\價量\上櫃增補資料')

append_dates =  pd.date_range('1994-01','2007-01',freq='M').sort_values(ascending=False)

url = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_print.php'

param = dict(l='zh-tw',s='0,asc,0')

for stock_id in table['代號'][401:]:
    param.update({'stkno':stock_id})
    save = root.joinpath(stock_id)
    save.mkdir(exist_ok=True)
    for i,date in enumerate(append_dates):
        print(stock_id,date.date())
        if save.joinpath(date.strftime('%Y-%m')).with_suffix('.html').exists():
            continue
        param.update({'d':date.strftime(f'{date.year-1911}/%m')})
        html = requests.get(url,param)
        assert html.status_code == 200
        time.sleep(10)
        if '共0筆' in html.text:
            break
        save.joinpath(date.strftime('%Y-%m')).with_suffix('.html').write_text(html.text,encoding='utf8')
    clear_output(wait=True)

70279 2006-12-31
70279 2006-11-30


In [68]:
root = pathlib.Path('D:\股票網頁\股票\價量\上櫃增補資料')

In [69]:
files = sorted(root.iterdir())

In [89]:
columns = ['日 期', '成交仟股', '成交仟元', '開盤', '最高', '最低', '收盤', '漲跌', '筆數']

for stock in files:
    stock_id = stock.name
    htmls = sorted(stock.glob('*.html'))
    
    data = pd.DataFrame()
    for html in htmls:

        month = pd.Timestamp(html.with_suffix('').name)
        month = month.strftime(f'{month.year-1911}/%m')
        print(stock_id,month)
        html = html.read_text(encoding='utf8')


        table = BeautifulSoup(html)
        info = table.find('tr').text

        assert month in info
        assert stock_id in info

        table.find('tr').decompose()
        table.find('tfoot').decompose()

        table = pd.read_html(table.prettify())[0]

        table = table.loc[(table[['開盤', '最高', '最低', '收盤']]!='--').all(axis=1)]
        
        if table.empty:
            continue
            
        table[['開盤', '最高', '最低', '收盤']] = table[['開盤', '最高', '最低', '收盤']].astype(float)
        assert table.columns.to_list() == columns
        assert (table.dtypes == ['object','int64','int64','float64','float64','float64','float64','float64','int64']).all()

        table[['成交仟股', '成交仟元']] = table[['成交仟股', '成交仟元']] * 1000

        table.columns = ['date', 'volume', 'capital', 'open', 'high', 'low', 'close', '漲跌', 'count']
        table['name'] = info.split()[2].replace('股票名稱:','')
        table['ID'] = stock_id
        table['market'] = 'otc'

        date = table.date.str.replace('＊','').str.split('/',expand=True).astype(int)
        date[0] = date[0]+1911
        date = date.astype(str)
        date.columns = ['year', 'month', 'day']
        table['date'] = pd.to_datetime(date)

        table = table.set_index('date')
        table = table[['open','high','low','close','volume','count','capital','ID','name','market']]
        
        data = data.append(table)
        clear_output(wait=True)
    
    data = data.sort_index()
    data = data.reset_index()
    data['date'] = data['date'].astype(str)
    
    data.to_csv(pathlib.Path(r'D:\股票網頁\股票\價量\上櫃增補資料_csv').joinpath(stock_id).with_suffix('.csv').as_posix(),index=False)

9962 95/12
